In [1]:
# Application of order reduction techniques to toy dataset prior to use with generative model
# Using toy dataset: sklearn cancer research
#
# Author: Alex Braafladt
#
# Version: v1 Initial creation 4/29/2022
#
# References:
#   -https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-wisconsin-diagnostic-dataset
#   -https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

In [2]:
# imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

# statistical functions
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.spatial.distance import jensenshannon
from scipy import stats as st
from scipy.stats import multiscale_graphcorr
from scipy.stats import pearsonr

# order reduction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA